<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span></li><li><span><a href="#Тестирование-моделей" data-toc-modified-id="Тестирование-моделей-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование моделей</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Общий вывод</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

Признаки:
- RowNumber — индекс строки в данных
- CustomerId — уникальный идентификатор клиента
- Surname — фамилия
- CreditScore — кредитный рейтинг
- Geography — страна проживания
- Gender — пол
- Age — возраст
- Tenure — сколько лет человек является клиентом банка
- Balance — баланс на счёте
- NumOfProducts — количество продуктов банка, используемых клиентом
- HasCrCard — наличие кредитной карты
- IsActiveMember — активность клиента
- EstimatedSalary — предполагаемая зарплата <br>
Целевой признак:
- Exited — факт ухода клиента

## Подготовка данных

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
import matplotlib.pyplot as plt
from sklearn import tree
import numpy as np
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from  sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [2]:
data = pd.read_csv('/datasets/Churn.csv')

In [3]:
data.head(10)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0
5,6,15574012,Chu,645,Spain,Male,44,8.0,113755.78,2,1,0,149756.71,1
6,7,15592531,Bartlett,822,France,Male,50,7.0,0.00,2,1,1,10062.80,0
7,8,15656148,Obinna,376,Germany,Female,29,4.0,115046.74,4,1,0,119346.88,1
8,9,15792365,He,501,France,Male,44,4.0,142051.07,2,0,1,74940.50,0
9,10,15592389,H?,684,France,Male,27,2.0,134603.88,1,1,1,71725.73,0


In [4]:
data.columns = data.columns.str.lower()
data.columns

Index(['rownumber', 'customerid', 'surname', 'creditscore', 'geography',
       'gender', 'age', 'tenure', 'balance', 'numofproducts', 'hascrcard',
       'isactivemember', 'estimatedsalary', 'exited'],
      dtype='object')

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rownumber        10000 non-null  int64  
 1   customerid       10000 non-null  int64  
 2   surname          10000 non-null  object 
 3   creditscore      10000 non-null  int64  
 4   geography        10000 non-null  object 
 5   gender           10000 non-null  object 
 6   age              10000 non-null  int64  
 7   tenure           9091 non-null   float64
 8   balance          10000 non-null  float64
 9   numofproducts    10000 non-null  int64  
 10  hascrcard        10000 non-null  int64  
 11  isactivemember   10000 non-null  int64  
 12  estimatedsalary  10000 non-null  float64
 13  exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


Наблюдаем пропуски только в столбце 'tenure'.

In [6]:
data.describe()

,rownumber,customerid,creditscore,age,tenure,balance,numofproducts,hascrcard,isactivemember,estimatedsalary,exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [7]:
data['tenure'].unique()

array([ 2.,  1.,  8.,  7.,  4.,  6.,  3., 10.,  5.,  9.,  0., nan])

Заметим, что все значения целые. Поэтому можем заменить тип данных на int. И заполним пропуски медианным значением (оно примерно равно среднему, но целое).

In [8]:
data['tenure'] = data['tenure'].fillna(data['tenure'].median())
data['tenure'] = data['tenure'].astype('int')

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rownumber        10000 non-null  int64  
 1   customerid       10000 non-null  int64  
 2   surname          10000 non-null  object 
 3   creditscore      10000 non-null  int64  
 4   geography        10000 non-null  object 
 5   gender           10000 non-null  object 
 6   age              10000 non-null  int64  
 7   tenure           10000 non-null  int64  
 8   balance          10000 non-null  float64
 9   numofproducts    10000 non-null  int64  
 10  hascrcard        10000 non-null  int64  
 11  isactivemember   10000 non-null  int64  
 12  estimatedsalary  10000 non-null  float64
 13  exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


Проверим дубликаты.

In [10]:
data.drop(['rownumber', 'customerid'], axis=1).duplicated().sum()

0

In [11]:
data[['surname', 'geography', 'gender', 'age']].duplicated().sum()

241

Есть неявные дубликаты. Удалим их.

In [12]:
data = data.drop_duplicates(['surname', 'geography', 'gender', 'age'])

In [13]:
data['geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [14]:
data['gender'].unique()

array(['Female', 'Male'], dtype=object)

Неявных дубликатов внутри столбцов с качественными категориями не обнаружено.

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9759 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   rownumber        9759 non-null   int64  
 1   customerid       9759 non-null   int64  
 2   surname          9759 non-null   object 
 3   creditscore      9759 non-null   int64  
 4   geography        9759 non-null   object 
 5   gender           9759 non-null   object 
 6   age              9759 non-null   int64  
 7   tenure           9759 non-null   int64  
 8   balance          9759 non-null   float64
 9   numofproducts    9759 non-null   int64  
 10  hascrcard        9759 non-null   int64  
 11  isactivemember   9759 non-null   int64  
 12  estimatedsalary  9759 non-null   float64
 13  exited           9759 non-null   int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


<b> Итог: пропуски заполнены, типы данных исправленны, дубликаты удалены.

## Исследование задачи

Обозначим признаки и целевой фактор. В признаках не будем учитывать индивидуальный номер и id клиента и его фамилию, так как они не являются показателем чего-либо. 

In [16]:
data = data.drop(['rownumber', 'customerid', 'surname'], axis=1)

Выделим целевой признак.

In [20]:
features = data.drop('exited', axis=1)
target = data['exited']

Разобьем нашу выборку на тренировочную, валидационную и тестовую в соотношении 3:1:1. При этом в каждой выборке сохраним пропорции классов.

In [21]:
features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.4, random_state=12345, stratify=target)
features_test, features_valid, target_test, target_valid = train_test_split(
    features_valid, target_valid, test_size=0.5, random_state=12345, stratify=target_valid)

In [22]:
print(target_train.shape)
print(target_valid.shape)
print(target_test.shape)

(5855,)
(1952,)
(1952,)


In [23]:
print(np.bincount(target_train))
print(np.bincount(target_valid))
print(np.bincount(target_test))

[4653 1202]
[1551  401]
[1551  401]


In [24]:
features_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5855 entries, 3440 to 2088
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   creditscore      5855 non-null   int64  
 1   geography        5855 non-null   object 
 2   gender           5855 non-null   object 
 3   age              5855 non-null   int64  
 4   tenure           5855 non-null   int64  
 5   balance          5855 non-null   float64
 6   numofproducts    5855 non-null   int64  
 7   hascrcard        5855 non-null   int64  
 8   isactivemember   5855 non-null   int64  
 9   estimatedsalary  5855 non-null   float64
dtypes: float64(2), int64(6), object(2)
memory usage: 503.2+ KB


Применим технико ohe и закодируем категориальные признаки

In [25]:
features_categirical = ['geography', 'gender']

ohe = OneHotEncoder(sparse=False, drop='first')
ohe.fit(features_train[features_categirical])

def features_ohe(ohe_variable, df_features, features_categ):
    df_features_ohe = pd.DataFrame(
        data=ohe_variable.transform(df_features[features_categ]), 
        index=df_features.index,
        columns=ohe_variable.get_feature_names()
    )

    df_features = df_features.drop(features_categ, axis=1)
    df_features = df_features.join(df_features_ohe)
    return df_features    

features_train = features_ohe(ohe, features_train, features_categirical);
features_valid = features_ohe(ohe, features_valid, features_categirical);
features_test = features_ohe(ohe, features_test, features_categirical);

Признаки имеют совершенно разные разбросы значений, поэтому проведем масштабирование.

In [26]:
numeric = ['creditscore', 'age', 'tenure', 'balance', 'numofproducts', 'estimatedsalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

<b>Обучим модель "случайный лес". <b> <br>Переберем гиперпараметры глубину и количество деревьев, чтобы найти оптимальную модель.

In [27]:
%%time
best_f1 = 0
best_est = 0
best_depth = 0
roc_auc_best=0
for est in range(5, 121, 5):
    for depth in range(4, 21):   
        model = RandomForestClassifier(random_state=12345, n_estimators=est, max_depth=depth)
        model.fit(features_train, target_train)
        predictions = model.predict(features_valid)
        f1 = f1_score(target_valid, predictions)
        if f1 > best_f1:
            best_f1 = f1
            best_est = est
            best_depth = depth
            probabilities = model.predict_proba(features_valid)
            probabilities_one_test = probabilities[:, 1]
            roc_auc_best = roc_auc_score(target_valid, probabilities_one_test)
print('est =', best_est, 'depth =', best_depth, 'f1=', best_f1, 'ROC-AUC:', roc_auc_best)

est = 35 depth = 18 f1= 0.6068759342301943 ROC-AUC: 0.8481520248379696
CPU times: user 2min 34s, sys: 751 ms, total: 2min 35s
Wall time: 2min 36s


f1 мера лучшей модели 0.6069. 

Показатель AUC-ROC больше f1-меры, значит модель в норме.

<b>Обучим модель "логистическая регрессия".

In [28]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
print('f1:', f1_score(target_valid, predictions))
probabilities = model.predict_proba(features_valid)
probabilities_one_test = probabilities[:, 1]
print('roc-auc:', roc_auc_score(target_valid, probabilities_one_test))

f1: 0.3217550274223035
roc-auc: 0.7637627401515554


Ее f1 мера слишком низкая для использования.

# <font size="3"><b> Вывод: было построено две модели. Модель "логистическая регрессия" имеет слишком низкую f1-меру, поэтому она нам не подходит. Самые лучшие показатели имеет модель "случайный лес" из 35 деревьев с максимальной глубиной 18. f1=0.61, AUC-ROC=0.85</b></font>

## Борьба с дисбалансом

In [29]:
np.bincount(target_train)

array([4653, 1202])

Заметим дисбаланс в количестве классов в обучающей выборке: класса 0 - 4653, а класса 1 - 1202. Поэтому используем два метода для уравновешивания классов. Сначала увеличим количество 1, затем уменьшим количество 0.

In [30]:
features_zeros = features_train[target_train == 0]
features_ones = features_train[target_train == 1]
target_zeros = target_train[target_train == 0]
target_ones = target_train[target_train == 1]

<B> Upsample

In [31]:
f1_best=0
best_estimators=0
best_depth=0
roc_auc_best=0
features_upsampled = pd.concat([features_zeros] + [features_ones] * 4)
target_upsampled = pd.concat([target_zeros] + [target_ones] * 4)
features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=12345)
for estimators in range(10, 81, 5):
    for depth in range(4, 21):
        model = RandomForestClassifier(random_state=12345, n_estimators=estimators, max_depth=depth)
        model.fit(features_upsampled, target_upsampled)
        predictions = model.predict(features_valid)
        f1 = f1_score(target_valid, predictions)
        if f1 > f1_best:
            f1_best = f1
            best_estimators = estimators
            best_depth = depth
            probabilities = model.predict_proba(features_valid)
            probabilities_one_test = probabilities[:, 1]
            roc_auc_best = roc_auc_score(target_valid, probabilities_one_test)
print('estimators:', best_estimators, '|', 'depth:', best_depth)
print('f1:', f1_best)        
print('ROC-AUC:', roc_auc_best)

estimators: 75 | depth: 10
f1: 0.6288308740068105
ROC-AUC: 0.8524755165599862


<B> Downsample

In [32]:
f1_best=0
best_estimators=0
best_depth=0
roc_auc_best=0
features_downsampled = pd.concat([features_zeros.sample(frac=0.25, random_state=12345)] + [features_ones])
target_downsampled = pd.concat([target_zeros.sample(frac=0.25, random_state=12345)] + [target_ones])
features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)
for estimators in range(10, 81, 5):
    for depth in range(4, 21):
        model = RandomForestClassifier(random_state=12345, n_estimators=estimators, max_depth=depth)
        model.fit(features_downsampled, target_downsampled)
        predictions = model.predict(features_valid)
        f1 = f1_score(target_valid, predictions)
        if f1 > f1_best:
            f1_best = f1
            best_estimators = estimators
            best_depth = depth
            probabilities = model.predict_proba(features_valid)
            probabilities_one_test = probabilities[:, 1]
            roc_auc_best = roc_auc_score(target_valid, probabilities_one_test)
print('estimators:', best_estimators, '|', 'depth:', best_depth)
print('f1:', f1_best)        
print('ROC-AUC:', roc_auc_best)

estimators: 50 | depth: 6
f1: 0.592156862745098
ROC-AUC: 0.8438381801781814


In [33]:
features_downsampled = pd.concat([features_zeros.sample(frac=0.75, random_state=12345)] + [features_ones])
target_downsampled = pd.concat([target_zeros.sample(frac=0.75, random_state=12345)] + [target_ones])
features_downsampled, target_downsampled = shuffle(features_downsampled, target_downsampled, random_state=12345)
model = RandomForestClassifier(random_state=12345, n_estimators=35, max_depth=18)
model.fit(features_downsampled, target_downsampled)
predictions = model.predict(features_valid)
print('f1:', f1_score(target_valid, predictions))
probabilities = model.predict_proba(features_valid)
probabilities_one_test = probabilities[:, 1]
print('ROC-AUC', roc_auc_score(target_valid, probabilities_one_test))

f1: 0.6100278551532033
ROC-AUC 0.8441693959813555


<b> Логистическая регрессия

Исправим дисбаланс классов параметром class_weight.

In [34]:
model = LogisticRegression(class_weight='balanced', random_state=12345)
model.fit(features_train, target_train)
predictions = model.predict(features_valid)
print('f1:', f1_score(target_valid, predictions))

f1: 0.49738219895287966


Исправим сэмплированием.

In [35]:
model = LogisticRegression(random_state=12345)
model.fit(features_upsampled, target_upsampled)
predictions = model.predict(features_valid)
print('f1:', f1_score(target_valid, predictions))

f1: 0.4927536231884057


In [36]:
model = LogisticRegression(random_state=12345)
model.fit(features_downsampled, target_downsampled)
predictions = model.predict(features_valid)
print('f1:', f1_score(target_valid, predictions))

f1: 0.40894568690095845


<b> Вывод: для борьбы с дисбалансом было применено 2 метода - upsample и downsample. Оба метода повысили f1-меру модели.

## Тестирование моделей

In [37]:
model = RandomForestClassifier(random_state=12345, n_estimators=75, max_depth=10)
model.fit(features_upsampled, target_upsampled)
predictions = model.predict(features_test)
f1_score(target_test, predictions)

0.5997770345596433

In [38]:
model = RandomForestClassifier(random_state=12345, n_estimators=50, max_depth=6)
model.fit(features_downsampled, target_downsampled)
predictions = model.predict(features_test)
f1_score(target_test, predictions)

0.5122349102773246

In [39]:
features_train = pd.concat([features_upsampled, features_valid])
target_train = pd.concat([target_upsampled, target_valid])
model = RandomForestClassifier(random_state=12345, n_estimators=75, max_depth=10)
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print('f1:', f1_score(target_test, predictions))
probabilities = model.predict_proba(features_test)
probabilities_one_test = probabilities[:, 1]
print('ROC-AUC', roc_auc_score(target_test, probabilities_one_test))

f1: 0.6011976047904191
ROC-AUC 0.8601192055322686


In [40]:
features_train = pd.concat([features_downsampled, features_valid])
target_train = pd.concat([target_downsampled, target_valid])
model = RandomForestClassifier(random_state=12345, n_estimators=50, max_depth=6)
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print('f1:', f1_score(target_test, predictions))
probabilities = model.predict_proba(features_test)
probabilities_one_test = probabilities[:, 1]
print('ROC-AUC', roc_auc_score(target_valid, probabilities_one_test))

f1: 0.5328947368421052
ROC-AUC 0.5046008447610825


<b> Модель "случайный лес" из 75 деревьев глубины 10 прошел проверку! <br>
   Все модели на которых использовалось downsample показали себя хуже.

## Общий вывод

<font size="3"><b> В этом проекте велась работа с данными о клиентах "Бета-банка". Главной целью было прогнозирование оттока клиентов (уйдет конкретный клиент или нет). Сначала была проделана предобработка данных: удаление дубликатов, исправление типов данных, заполнение пропусков. Затем были проанализированы несколько моделей. Самый лучший результат показала модель "слцчайный лес" с f1-мерой равной 0.60. Но в данных присутствовал большой дисбаланс в классах. Он был удален двумя методами, оба показали улучшение модели. Самой лучшей оказалась: модель "случайный лес" из 75 деревьев глубины 10. </b></font>